In [4]:
import pandas as pd
import requests
import time
import utils as eda
import os

## Datasets de peliculas y datos de reseñas y etiquetas

In [5]:

print(os.listdir("../data/interim/movies-data/ml-32m"))

['README.txt', 'checksums.txt', 'tags.csv', 'movies.csv', 'links.csv', 'ratings.csv']


In [ ]:

informacion = ["links", "movies", "ratings", "tags"]
df_list = []

for i in informacion:
    df = pd.read_csv(f"../data/interim/movies-data/ml-32m/{i}.csv")
    df_list.append(df)


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

: 

In [ ]:
#Links
links_df = df_list[0].head()


##La siguiente información muestra como funciona la info que sale en el dataset de links
"""row = df_list[df_list["movieId"] == 1].iloc[0]

imdb_url = f"https://www.imdb.com/title/tt{int(row['imdbId']):07d}"
tmdb_url = f"https://www.themoviedb.org/movie/{int(row['tmdbId'])}"

print("IMDb:", imdb_url)
print("TMDb:", tmdb_url"""

links_df

In [ ]:
#Movies 
movies_df = df_list[1]
movies_df.head()

In [ ]:
#Raiting
rating_df =  df_list[2].head()
rating_df

In [ ]:
#Tags
tags_df = df_list[3].head()
tags_df

## Información referente a comentarios de sentimientos, estos deben asociarse a peliculas 

In [ ]:
def cargar_reseñas(directorio):
    textos = []
    etiquetas = []

    for label in ['pos', 'neg']:
        path = os.path.join(directorio, label)
        for archivo in os.listdir(path):
            with open(os.path.join(path, archivo), encoding='utf-8') as f:
                textos.append(f.read())
                etiquetas.append(1 if label == 'pos' else 0)
    return textos, etiquetas

# Cargar datos de entrenamiento y prueba
train_texts, train_labels = cargar_reseñas("../data/interim/feeling/aclImdb/train") ##texto de reseñas // ETIQUETA para positivo y negativo
test_texts, test_labels = cargar_reseñas("../data/interim/feeling/aclImdb/test")


## Otro sitio de peliculas

Este servira mas para mostrar datos o complementar los de una pelicula que ya tengamos en el dataset principal

In [ ]:
##Esta es para varias peliculas
api_key = "fa1b7162" ##Esto es lo importante, no lo borren ni cambien
peliculas = ["Inception", "The Matrix", "Toy Story"]

datos_peliculas = []

for titulo in peliculas:
    url = f"http://www.omdbapi.com/?t={titulo}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    if data["Response"] == "True":
        datos_peliculas.append(data)

datos_peliculas

In [ ]:
df = pd.DataFrame(datos_peliculas, columns=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production', 'Website', 'Response'])
print(df)

Combinación de ambas

In [ ]:
movies_df.size

In [ ]:
import re
movies = df_list[1].head(50)
# Función para limpiar título
def limpiar_titulo(titulo):
    # Esto elimina " (1995)" u otros años entre paréntesis
    return re.sub(r"\s*\(\d{4}\)$", "", titulo)

# Aplicar limpieza
movies["clean_title"] = movies["title"].apply(limpiar_titulo)

# Extraer año si quieres
def extraer_ano(titulo):
    match = re.search(r"\((\d{4})\)$", titulo)
    if match:
        return int(match.group(1))
    return None

movies["year"] = movies["title"].apply(extraer_ano)

movies.head()

In [ ]:

omdb_url = "http://www.omdbapi.com/"

# 3️⃣ Función para obtener info de OMDb
def obtener_info_omdb(titulo):
    params = {
        "t": titulo,
        "apikey": api_key,
        "plot": "short",  # puedes poner "full" si quieres la sinopsis completa
        "r": "json"
    }
    response = requests.get(omdb_url, params=params)
    data = response.json()
    if data.get("Response") == "True":
        return {
            "Title": data.get("Title"),
            "Year": data.get("Year"),
            "Genre": data.get("Genre"),
            "Director": data.get("Director"),
            "Actors": data.get("Actors"),
            "imdbRating": data.get("imdbRating"),
            "Plot": data.get("Plot")
        }
    else:
        return None

# 4️⃣ Consultar OMDb para todas las películas y guardar resultados
info_peliculas = []
for index, row in movies.iterrows():
    titulo = row["clean_title"]
    info = obtener_info_omdb(titulo)
    if info:
        info["movieId"] = row["movieId"]
        info_peliculas.append(info)
    time.sleep(0.2)  # para no saturar la API (limite de peticiones)

# 5️⃣ Crear DataFrame final
df_omdb = pd.DataFrame(info_peliculas)
##df_omdb.to_csv("movies_omdb.csv", index=False) Esto guarda un csv asi que hay que ver una ruta en caso de que vayamos a usarlo

df_omdb.head()

## Dataset de Amazon de reseñas

In [ ]:
import pandas as pd
import gzip

file_path = "../data/interim/movies.txt.gz"
data = []
current_review = {}
max_reviews = 10000  # cambiar según lo que quieras cargar

with gzip.open(file_path, "rt", encoding="latin-1") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        if line.startswith("product/productId:"):
            if current_review:
                data.append(current_review)
                if len(data) >= max_reviews:
                    break  # paramos al llegar al límite
                current_review = {}
            current_review["productId"] = line[len("product/productId: "):]
        elif line.startswith("review/userId:"):
            current_review["userId"] = line[len("review/userId: "):]
        elif line.startswith("review/profileName:"):
            current_review["profileName"] = line[len("review/profileName: "):]
        elif line.startswith("review/helpfulness:"):
            current_review["helpfulness"] = line[len("review/helpfulness: "):]
        elif line.startswith("review/score:"):
            try:
                current_review["score"] = float(line[len("review/score: "):])
            except:
                current_review["score"] = None
        elif line.startswith("review/time:"):
            try:
                current_review["time"] = int(line[len("review/time: "):])
            except:
                current_review["time"] = None
        elif line.startswith("review/summary:"):
            current_review["summary"] = line[len("review/summary: "):] if len(line) > len("review/summary: ") else ""
        elif line.startswith("review/text:"):
            current_review["text"] = line[len("review/text: "):] if len(line) > len("review/text: ") else ""

    if current_review and len(data) < max_reviews:
        data.append(current_review)

df = pd.DataFrame(data)
print(df.head())
print(f"Se cargaron {len(df)} reseñas")



In [ ]:
df.head(10)

## Datasets de Reseñas

tiene reseñas y se clasifican en sentimiento, pero no estan asociados a peliculas asi que las asignamos aleatoriamente

In [ ]:
csv_path = '../data/interim/imdb-dataset/IMDB Dataset SPANISH.csv'

df_imdb = pd.read_csv(csv_path)
df_imdb.head()

In [ ]:
import numpy as np

# Se le asigna un id del dataset de peliculas

# Número de reseñas
n_reviews = len(df_imdb)

# Lista de IDs de películas disponibles
movie_ids = movies_df['movieId'].tolist()

# Asignar aleatoriamente un movieId a cada reseña
df_imdb['movieId'] = np.random.choice(movie_ids, size=n_reviews)

# Revisar
df_imdb.head()